In [ ]:
## imports ##
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import MFTime
import MFParse
import os
import scipy.signal
import seaborn as sns
####

## global ##
dataPath='/Users/ziegler/repos/mayfly/output/timeSeries1252021.pkl'
simPath='/Users/ziegler/p8/egg_files/eggsTemplates1252021/'
templatePitchAngles=np.linspace(85,90,51)
templatePos=np.linspace(0,5e-2,21)
radius=0.00
#nPeaks=5
nTest=100
noiseVar=2.3e-14
colors=['r','b','g','c','m','k']
frequencyConversion=200e6/8192
array_rad=0.1
Vrange=(80-280*array_rad+300*array_rad**2)*1e-9
nSlice=2
sliceSize=8192
samplePeriod=5e-9
time=np.arange(0,sliceSize)
####

## definitions ##

def getEggFilePaths(dirPath,listOfPaths):
    try:
        os.path.isdir(dirPath)
    except:
        return False

    for fPath in os.listdir(dirPath):
        if fPath.split('.')[-1]=='egg':
            listOfPaths.append(os.path.join(dirPath,fPath))

    return True

def getEggFileParams(eggPath,params):
    try:
        os.path.isfile(eggPath)
    except:
        return False

    try:
        pitchAngle=float(eggPath.split('/')[-1].split('Angle')[-1].split('_')[0])
        radius=float(eggPath.split('/')[-1].split('Pos')[-1].split('.egg')[0])
    except:
        return False

    params.append(pitchAngle)
    params.append(radius)

    return True

def getSimpleTemplate(frequencies,amplitudes,relPhase,nSample,dT):

    simpleTemplate=np.zeros(nSample,dtype=np.complex128)
    time=np.arange(0,nSample,1)*dT
    maxAmplitude=np.argmax(amplitudes)
    for i in range(len(frequencies)):
        if i == maxAmplitude:
            simpleTemplate+=amplitudes[i]*np.exp(+1j*2*np.pi*(frequencies[i]*time))
        else:
            simpleTemplate+=amplitudes[i]*np.exp(+1j*2*np.pi*(frequencies[i]*time+relPhase))

    return simpleTemplate

def addNoise(input,var):

    noise=np.random.multivariate_normal([0,0],np.eye(2)*var/2,len(input))
    noise=noise[:,0]+1j*noise[:,1]

    return input+noise

def getSimpleFrequencyEstimate(frequencySeries,threshold=0.25):

    maxValue=np.max(abs(frequencySeries))
    maxInd=np.argmax(abs(frequencySeries))
    #print(maxInd)

    rawPeakInds=np.where(abs(frequencySeries)>threshold*maxValue)[0]
    
    bigPeakInd=np.flip(np.argsort(abs(frequencySeries)))[0:10]
    bigPeakAmp=abs(frequencySeries[bigPeakInd])
    
    groupedPeakIndInd=np.argsort(bigPeakInd)
    
    groupedPeakInd=bigPeakInd[groupedPeakIndInd]
    groupedPeakAmp=bigPeakAmp[groupedPeakIndInd]

    return groupedPeakInd,groupedPeakAmp

####


In [ ]:
eggPaths=[]
badEgg=[]

data={'radius':[],'pitch-angle':[],'dist-type':[],'mf_output':[]}
radii=np.array([0.0])
pa=[]
if getEggFilePaths(simPath,eggPaths): # put all the found egg files into a list
    #print('test')
    for eggPath in eggPaths: # iterate over the simulation files

        simulationParams=[]
        getEggFileParams(eggPath,simulationParams)
        #print(abs(radii-simulationParams[1]))
        
        if simulationParams[1] not in radii:
            continue
            
        print(simulationParams)

        eggDataTime=MFParse.parse_egg(eggPath,Vrange=Vrange)
        eggSliceTime=MFParse.slice_egg(eggDataTime,nSlice,sliceSize)

        if MFParse.check_egg_slice(eggSliceTime): # check if the simulation is broken
            badEgg.append(eggPath)
            print(eggPath)
        else:
            tempDict={}

            eggSliceFFT,FFTFrequencies=MFTime.katydid_fft(eggSliceTime)
            trueSignalTime=eggSliceTime[0,:]
            trueSignalFreq=eggSliceFFT[0,:]/sliceSize
        
            freqInd,freqAmp=getSimpleFrequencyEstimate(trueSignalFreq)

            freq=FFTFrequencies[freqInd]
               
            
            for n in range(nTest):
            
                noisyData=addNoise(trueSignalTime,noiseVar)
                noisyDataFFT=np.fft.fftshift(np.fft.fft(noisyData))
                justNoise=addNoise(np.zeros(trueSignalTime.size,dtype=np.complex128),noiseVar)
                justNoiseFFT=np.fft.fftshift(np.fft.fft(justNoise))

                phases=np.linspace(0,1,10)
                
                ####

                innerProduct=np.zeros(phases.size)
                
                for i,relPhase in enumerate(phases):

                    simpleTemplate=getSimpleTemplate(freq,freqAmp,relPhase,sliceSize,samplePeriod)
                    simpleTemplateFFT=np.fft.fftshift(np.fft.fft(simpleTemplate))
                    #innerProduct[i]=abs(scipy.signal.fftconvolve(simpleTemplate,noisyData,mode='valid'))
                    innerProduct[i]=abs(np.sum(noisyDataFFT*simpleTemplateFFT)/8192)
                    
                mfSignal=np.max(innerProduct)
                
                data['pitch-angle'].append(simulationParams[0])
                data['radius'].append(simulationParams[1])
                data['dist-type'].append('simple')
                data['mf_output'].append(mfSignal)
                
                innerProduct=np.zeros(phases.size)
                
                for i,relPhase in enumerate(phases):

                    simpleTemplate=getSimpleTemplate(freq,freqAmp,relPhase,sliceSize,samplePeriod)
                    simpleTemplateFFT=np.fft.fftshift(np.fft.fft(simpleTemplate))
                    #innerProduct[i]=abs(scipy.signal.fftconvolve(simpleTemplate,justNoise,mode='valid'))
                    innerProduct[i]=abs(np.sum(justNoiseFFT*simpleTemplateFFT)/8192)
                
                mfNoise=np.max(innerProduct)
                
                data['pitch-angle'].append(simulationParams[0])
                data['radius'].append(simulationParams[1])
                data['dist-type'].append('simple-noise')
                data['mf_output'].append(mfNoise)
                
                ####
                
                mfOmni=np.max(abs(scipy.signal.fftconvolve(trueSignalTime,noisyData,mode='full')))
                
                data['pitch-angle'].append(simulationParams[0])
                data['radius'].append(simulationParams[1])
                data['dist-type'].append('omni')
                data['mf_output'].append(mfOmni)
                
                mfOmniNoise=np.max(abs(scipy.signal.fftconvolve(trueSignalTime,justNoise,mode='full')))
                
                data['pitch-angle'].append(simulationParams[0])
                data['radius'].append(simulationParams[1])
                data['dist-type'].append('omni-noise')
                data['mf_output'].append(mfOmniNoise)
                
                
            #plt.plot(8192*abs(np.fft.ifft(np.fft.ifftshift(simpleTemplateFFT*trueSignalFreq))))
           
                #data['pitch-angle'].append(simulationParams[0])
                #data['radius'].append(simulationParams[1])
                #data['simple'].append(mfSignal)
                #data['noise-simple'].append(mfNoise)
                #data['omni'].append(mfOmni)
                #data['noise-omni'].append(mfOmniNoise)
            #plt.figure()
            #plt.hist(mfSignal,10)
            #plt.hist(mfNoise,10)
            #plt.show()
            
            
            #input('press enter')


In [ ]:
datapd=pd.DataFrame(data)
datasimple=datapd[datapd['dist-type'].isin(['simple','simple-noise'])]
dataomni=datapd[datapd['dist-type'].isin(['omni','omni-noise'])]
fig1=plt.figure()
sns.set_theme(style='whitegrid')
palette=sns.color_palette('Set2',n_colors=4)
#sns.set_palette(sns.color_palette('pastel',n_colors=2))
ax1=sns.lineplot(x='pitch-angle',y='mf_output',data=datapd,ci='sd',
                 hue='dist-type',palette=palette)
#sns.lineplot(x='pitch-angle',y='omni',data=datapd,ci='sd')
#ax1=sns.lineplot(x='pitch-angle',y='noise-omni',data=datapd,ci='sd',hue='simple')
#ax1=sns.lineplot(x='pitch-angle',y='noise-omni',data=datapd,ci='sd')

ax1.set_xlabel('Pitch Angle')
ax1.set_ylabel('MF Output')
ax1.legend(title='Distribution')
ax1.set_title('Matched Filter Outputs for Simple and Omniscient Templates\n R = 4 cm')

plt.savefig('/Users/ziegler/repos/romulus/output/MFOutputR4cmBoth.png')


fig2=plt.figure()
sns.set_theme(style='whitegrid')

ax2=sns.lineplot(x='pitch-angle',y='mf_output',
                 data=datasimple,ci='sd',hue='dist-type',palette=palette[0:2])

ax2.set_xlabel('Pitch Angle')
ax2.set_ylabel('MF Output')
ax2.legend(title='Distribution')
ax2.set_title('Matched Filter Outputs for Simple Template\n R = 4 cm')

plt.savefig('/Users/ziegler/repos/romulus/output/MFOutputR4cmSimple.png')

fig3=plt.figure()
sns.set_theme(style='whitegrid')

ax3=sns.lineplot(x='pitch-angle',y='mf_output',
                 data=dataomni,ci='sd',hue='dist-type',palette=palette[2:])

ax3.set_xlabel('Pitch Angle')
ax3.set_ylabel('MF Output')
ax3.legend(title='Distribution')
ax3.set_title('Matched Filter Outputs for Omniscient Template\n R = 4 cm')

plt.savefig('/Users/ziegler/repos/romulus/output/MFOutputR4cmOmni.png')


In [ ]:
ax2

In [ ]:
plt.figure()

plt.hist(datapd[datapd['pitch-angle']==87.0]['omni'],label='omni-signal')
plt.hist(datapd[datapd['pitch-angle']==87.0]['simple'],label='simple-signal')
plt.hist(datapd[datapd['pitch-angle']==87.0]['noise-simple'],label='simple-noise')
plt.legend(loc=0)
plt.xlabel('MF Output')
plt.ylabel('Counts')
plt.title('Histogram of MF Outputs for Simple Template\n and Omniscient Template on Noisy Data')
plt.savefig('/Users/ziegler/repos/romulus/output/2252021histogramsMFOutputs.png')
#np.std(datapd[datapd['pitch-angle']==90]['noise-simple'])

In [ ]:
print(sns.color_palette('deep',n_colors=4))